---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

# Datasets

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
 # State names Extraction
    U_States=pd.read_csv('university_towns.txt',sep='\n',header=None)[0]
    states_drop=U_States[U_States.str.contains('edit')] # States extraction: 'State[edit]'
    states=states_drop.map(lambda s: s[:-6])            # drop '[edit]'
    states.name='State'
    States=pd.Series(np.zeros(U_States.size),name="State").map(lambda x: np.nan)
    States[states.index]=states
    States.ffill(inplace=True)
    #print(States)

    # Town Name Extraction
    U_Towns=pd.read_csv('university_towns.txt',sep='\n',header=None).drop(states_drop.index)[0].str.split(' \(').map(lambda r: r[0])
    U_Towns.name='RegionName'
    #print(U_Towns)

    # University Towns DataFrame
    U_df=pd.DataFrame([States,U_Towns]).T.drop(states_drop.index)#.set_index(['State','RegionName'])
    return U_df

In [4]:
#get_list_of_university_towns()

In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
  
    def get_GDP():
        ## GDP Dataset
        GDP=pd.read_excel('gdplev.xls',header=5,parse_cols='E:G').reset_index()
        GDP=GDP.rename(columns={str(GDP.columns[0]):'Quarter'}).set_index('Quarter')
        GDP=GDP.iloc[GDP.index.get_loc('2000q1'):]   
        GDP.columns=GDP.columns.map(lambda c: re.sub(r'[^a-zA-Z\s209]+',"",c))
        return GDP
    
    df=get_GDP().drop(get_GDP().columns[0],axis=1)
    rec=[]
    index=0
    while index < (len(df)-2):
        q0=df.iloc[index].values[0]
        q1=df.iloc[index+1].values[0]
        q2=df.iloc[index+2].values[0]
    ## Para determinar el inicio de una recesión solo se compara el ciclo actual y los dos siguientes:    
        if(q1<q0 and q2<q1):
            rec.append(index)
        index+=3
        
    return df.iloc[rec].iloc[0].name

In [6]:
#get_recession_start()

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    def get_GDP():
        ## GDP Dataset
        GDP=pd.read_excel('gdplev.xls',header=5,parse_cols='E:G').reset_index()
        GDP=GDP.rename(columns={str(GDP.columns[0]):'Quarter'}).set_index('Quarter')
        GDP=GDP.iloc[GDP.index.get_loc('2000q1'):]   
        GDP.columns=GDP.columns.map(lambda c: re.sub(r'[^a-zA-Z\s209]+',"",c))
        return GDP
    
    df=get_GDP().drop(get_GDP().columns[0],axis=1)
    index=df.index.get_loc(get_recession_start())
    rec=[]
    while index < (len(df)-2):
        q0=df.iloc[index].values[0]
        q1=df.iloc[index+1].values[0]
        q2=df.iloc[index+2].values[0]
        if(q1>q0 and q2>q1): 
            rec.append(index)
        index+=3
        
    return df.iloc[rec].iloc[0].name

In [8]:
#get_recession_end()

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    def get_GDP():
        ## GDP Dataset
        GDP=pd.read_excel('gdplev.xls',header=5,parse_cols='E:G').reset_index()
        GDP=GDP.rename(columns={str(GDP.columns[0]):'Quarter'}).set_index('Quarter')
        GDP=GDP.iloc[GDP.index.get_loc('2000q1'):]   
        GDP.columns=GDP.columns.map(lambda c: re.sub(r'[^a-zA-Z\s209]+',"",c))
        return GDP
    
    df=get_GDP().drop(get_GDP().columns[0],axis=1)
    bottom_index=df.index.get_loc(get_recession_end())-2
    
    return df.iloc[bottom_index].name

In [10]:
#get_recession_bottom()

In [11]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    def get_house_prices():
        houses=pd.read_csv('City_Zhvi_AllHomes.csv')
        houses=houses.set_index(['State']).rename(states).set_index('RegionName',append=True).sort_index()
        return houses
    
    df=pd.DataFrame.copy(get_house_prices())
    start_quarter=df.columns.get_loc('2000-01')
    df_quarter=df.iloc[:,start_quarter:].groupby(np.arange( len(df.iloc[:,start_quarter:].columns) )//3,axis=1).mean()
    
    init_year=int(df.iloc[:,start_quarter].name[:4])
    last_year=int(df.iloc[:,-1].name[:4]) 
    
    df_quarter.columns= [ str(y)+q for y in range(init_year,last_year+1) for q in ['q1','q2','q3','q4'] ] [:-1] #do not return 2016q4
    return df_quarter

In [12]:
#convert_housing_data_to_quarters()

In [13]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Recession Period indexes
    start = convert_housing_data_to_quarters().columns.get_loc(get_recession_start())
    bottom = convert_housing_data_to_quarters().columns.get_loc(get_recession_bottom())
    
    # House Prices by Quarters DF (only recession period+prior quarter)
    house_recession_quarters = convert_housing_data_to_quarters().iloc[:,start-1:bottom+1]
    
    # Non University Towns Houses Prices (only recession period+prior quarter)
    non_U_Towns = house_recession_quarters.drop(get_list_of_university_towns()['RegionName'],level=1)
    
    # University Towns Houses Prices (only recession period)
    U_Towns = get_list_of_university_towns().set_index(['State','RegionName']).merge( house_recession_quarters ,how='inner',left_index=True,right_index=True)

    # T-Test (p<0.01 signifcance level)
        # Alternative hypothesis: non-U and U Towns are the same (both affected by recession)
        # Null hypothesis: non-U and U Towns are the differente (U Towns less affected by recession)
        
    price_ratio_U = U_Towns.iloc[:,0]/U_Towns.iloc[:,bottom-start+1]
    price_ratio_non_U = non_U_Towns.iloc[:,0]/non_U_Towns.iloc[:,bottom-start+1] 
    [t,p]=ttest_ind(price_ratio_U,price_ratio_non_U,nan_policy='omit')
    return (p<0.01,p,'university town' if price_ratio_U.mean()<price_ratio_non_U.mean() else 'non-university town')

In [14]:
#run_ttest()